In [1]:
import pandas as pd 
import numpy as np 

In [2]:
# load the data into dataframes
data_search_trend_US = pd.read_csv("/Users/chengchen/Documents/GitHub/Analyzing-COVID19-Search-Trends-and-Hospitalization/2020_US_weekly_symptoms_dataset.csv")
data_hospitalization = pd.read_csv("/Users/chengchen/Documents/GitHub/Analyzing-COVID19-Search-Trends-and-Hospitalization/hospitalization_dataset.csv")

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#remove symptoms that have no search data
data_search_trend_US.dropna(thresh=0, axis=1)
del data_search_trend_US['sub_region_2']
del data_search_trend_US['sub_region_2_code']
data_search_trend_US = data_search_trend_US.fillna(0)

In [4]:
#get the total number of regions 
regions = {}
for index,content in data_search_trend_US.iterrows():
    if content['sub_region_1_code'] not in regions:
        regions.update({content['sub_region_1_code']: 1})   

In [5]:
#get the hospitalization in the USA
data_hospitalization_US = data_hospitalization[data_hospitalization['open_covid_region_code'].isin(list(regions.keys()))]

In [6]:
#remove redundant features
data_hospitalization_US = data_hospitalization_US[['open_covid_region_code','region_name','date','hospitalized_cumulative','hospitalized_new']]

In [7]:
#reset the index
data_hospitalization_US = data_hospitalization_US[(data_hospitalization_US.T != 0).any()].reset_index()

In [8]:
#set index to date index
data_hospitalization_US['Date'] = pd.to_datetime(data_hospitalization_US['date'])
data_hospitalization_US = data_hospitalization_US.set_index('Date')
data_hospitalization_US = data_hospitalization_US.drop(['date'], axis=1)

In [9]:
# add a new column 
data_search_trend_US.loc[:,'hospitalized_new'] = 0

In [10]:
#change the value of hospitalized_new at the correct cell
for region in regions: 
    data_to_append = data_hospitalization_US[data_hospitalization_US['open_covid_region_code']==region]['hospitalized_new']
    data_to_append = data_to_append.resample('W', label='left', loffset=pd.DateOffset(days=1)).sum()
    for index,values in data_to_append.iteritems():
        if data_search_trend_US[(data_search_trend_US['date']==str(index.date()))&(data_search_trend_US['open_covid_region_code']==region)].index.values.astype(int).size==0:
            continue
        ind = data_search_trend_US[(data_search_trend_US['date']==str(index.date()))&(data_search_trend_US['open_covid_region_code']==region)].index.values.astype(int)[0]
        data_search_trend_US.at[ind,'hospitalized_new'] = values
        

In [11]:
del data_search_trend_US['sub_region_1_code']
del data_search_trend_US['Unnamed: 0']

In [12]:
data_search_trend_US.shape

(608, 428)

In [13]:
data_search_trend_US = data_search_trend_US.loc[:, (data_search_trend_US==0).mean() < .8]

In [14]:
#remove rows and cols that has over 80% of zero entries
data_search_trend_US = data_search_trend_US[data_search_trend_US.astype('bool').mean(axis=1)>=0.2]

In [15]:
data_search_trend_US.head()

,open_covid_region_code,country_region_code,country_region,sub_region_1,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amenorrhea,symptom:Angular cheilitis,...,symptom:Tachypnea,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Ventricular fibrillation,symptom:Viral pneumonia,hospitalized_new
0,US-AK,US,United States,Alaska,2020-01-06,12.69,0.00,0.0,12.29,0.00,...,7.93,14.28,15.86,11.90,18.84,13.88,0.00,9.91,0.0,0
1,US-AK,US,United States,Alaska,2020-01-13,9.56,0.00,0.0,12.62,0.00,...,10.71,13.39,12.24,7.84,16.83,13.01,7.65,8.41,0.0,0
2,US-AK,US,United States,Alaska,2020-01-20,0.00,0.00,0.0,14.17,0.00,...,12.33,12.88,17.66,15.27,18.40,12.14,7.54,16.19,0.0,0
3,US-AK,US,United States,Alaska,2020-01-27,15.31,7.47,0.0,15.67,7.47,...,10.02,13.30,12.76,9.84,20.41,17.49,0.00,0.00,0.0,0
4,US-AK,US,United States,Alaska,2020-02-03,8.81,0.00,0.0,13.87,8.06,...,18.18,11.81,16.68,10.68,20.62,16.87,0.00,10.68,10.5,0


In [16]:
columns = list(data_search_trend_US.iloc[0:,5:107])
for col in columns: 
    mean = data_search_trend_US[col].mean(axis=0)
    for index, row in data_search_trend_US.iterrows():
        new_val = (row[col] - mean)/mean
        data_search_trend_US.at[index, col] = new_val
    

In [17]:
data_search_trend_US.head()

,open_covid_region_code,country_region_code,country_region,sub_region_1,date,symptom:Adrenal crisis,symptom:Ageusia,symptom:Allergic conjunctivitis,symptom:Amenorrhea,symptom:Angular cheilitis,...,symptom:Tachypnea,symptom:Tenderness,symptom:Thyroid nodule,symptom:Trichoptilosis,symptom:Upper respiratory tract infection,symptom:Urethritis,symptom:Urinary urgency,symptom:Ventricular fibrillation,symptom:Viral pneumonia,hospitalized_new
0,US-AK,US,United States,Alaska,2020-01-06,0.789491,-1.000000,-1.0,1.123680,-1.000000,...,0.285116,1.334010,2.137488,1.222191,2.634172,2.047318,-1.000000,0.056126,-1.000000,0
1,US-AK,US,United States,Alaska,2020-01-13,0.348112,-1.000000,-1.0,1.180703,-1.000000,...,0.735636,1.188543,1.421365,0.464032,2.246449,1.856312,-0.016027,-0.103732,-1.000000,0
2,US-AK,US,United States,Alaska,2020-01-20,-1.000000,-1.000000,-1.0,1.448539,-1.000000,...,0.998169,1.105186,2.493571,1.851501,2.549297,1.665305,-0.030175,0.725397,-1.000000,0
3,US-AK,US,United States,Alaska,2020-01-27,1.158953,0.302886,-1.0,1.707735,0.028579,...,0.623816,1.173833,1.524233,0.837509,2.937019,2.839884,-1.000000,-1.000000,-1.000000,0
4,US-AK,US,United States,Alaska,2020-02-03,0.242350,-1.000000,-1.0,1.396700,0.109818,...,1.946205,0.930298,2.299703,0.994370,2.977528,2.703765,-1.000000,0.138186,0.338036,0


In [18]:
data_search_trend_US.to_csv('/Users/chengchen/Desktop/merged_dataset.csv')